# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hotan,37.0997,79.9269,13.58,22,13,1.70,CN,1697233751
1,1,cabo san lucas,22.8909,-109.9124,33.12,66,2,5.23,MX,1697233751
2,2,bikenibeu village,1.3673,173.1241,29.98,74,40,2.06,KI,1697233751
3,3,ladario,-19.0047,-57.6017,30.05,42,0,2.06,BR,1697233752
4,4,adamstown,-25.0660,-130.1015,20.99,79,100,7.16,PN,1697233719


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    alpha = .5,
    scale = .5,
    frame_width = 770,
    frame_height = 550,
    color = "City"
)

# Display the map plot_1
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down to the my ideal city: Max Temp 20-30C, Humidity < 80%, Cloudiness < 50% 
city_ideal_df = city_data_df[(city_data_df["Max Temp"] >= 30) & (city_data_df["Max Temp"] >= 20) & (city_data_df["Humidity"] <= 80) & (city_data_df["Cloudiness"] <=25) & (city_data_df["Wind Speed"] <=7)]

# Drop any rows with null values
city_ideal_df.dropna()

# Display sample data
display(len(city_ideal_df))
city_ideal_df

17

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,cabo san lucas,22.8909,-109.9124,33.12,66,2,5.23,MX,1697233751
3,3,ladario,-19.0047,-57.6017,30.05,42,0,2.06,BR,1697233752
17,17,djibo,14.1022,-1.6306,34.03,25,9,1.99,BF,1697233755
47,47,samana,19.2056,-69.3369,31.12,70,20,3.09,DO,1697233516
54,54,anton lizardo,19.0500,-95.9833,34.12,79,20,2.93,MX,1697233763
57,57,wailua homesteads,22.0669,-159.3780,30.07,62,0,4.63,US,1697233764
151,151,kapa'a,22.0752,-159.3190,30.82,62,0,4.63,US,1697233785
157,157,kongoussi,13.3258,-1.5347,31.86,31,3,1.10,BF,1697233787
198,198,itaituba,-4.2761,-55.9836,37.09,47,20,1.08,BR,1697233505
228,228,mabaruma,8.2000,-59.7833,30.00,74,20,3.09,GY,1697233803


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
display(hotel_df)

,City,Country,Lat,Lng,Humidity,Hotel Name
1,cabo san lucas,MX,22.8909,-109.9124,66,
3,ladario,BR,-19.0047,-57.6017,42,
17,djibo,BF,14.1022,-1.6306,25,
47,samana,DO,19.2056,-69.3369,70,
54,anton lizardo,MX,19.0500,-95.9833,79,
57,wailua homesteads,US,22.0669,-159.3780,62,
151,kapa'a,US,22.0752,-159.3190,62,
157,kongoussi,BF,13.3258,-1.5347,31,
198,itaituba,BR,-4.2761,-55.9836,47,
228,mabaruma,GY,8.2000,-59.7833,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
limit = 20
radius = 100000
categories = "accommodation.hotel"
params = {
    "limit":limit,
    "apiKey":geoapify_key, 
    "categories":categories
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    #ethnicity = types_df.loc[index, "ethnicity"]
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
ladario - nearest hotel: Hotel Rios
djibo - nearest hotel: No hotel found
samana - nearest hotel: Hotel Cotubanamá
anton lizardo - nearest hotel: Hotel CocoAventura
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
kapa'a - nearest hotel: Pono Kai Resort
kongoussi - nearest hotel: Hotel La Sagesse Annexe
itaituba - nearest hotel: TG Hotel
mabaruma - nearest hotel: Monty Brooms Hotel
nieuw amsterdam - nearest hotel: Leonsberg Lodge
mazatlan - nearest hotel: Mona Inn
lihue - nearest hotel: Kauai Palms
el matama - nearest hotel: الكوثر
cuatro cienegas de carranza - nearest hotel: No hotel found
ar ruways - nearest hotel: No hotel found
moussoro - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
1,cabo san lucas,MX,22.8909,-109.9124,66,Comfort Rooms
3,ladario,BR,-19.0047,-57.6017,42,Hotel Rios
17,djibo,BF,14.1022,-1.6306,25,No hotel found
47,samana,DO,19.2056,-69.3369,70,Hotel Cotubanamá
54,anton lizardo,MX,19.0500,-95.9833,79,Hotel CocoAventura
57,wailua homesteads,US,22.0669,-159.3780,62,Hilton Garden Inn Kauai Wailua Bay
151,kapa'a,US,22.0752,-159.3190,62,Pono Kai Resort
157,kongoussi,BF,13.3258,-1.5347,31,Hotel La Sagesse Annexe
198,itaituba,BR,-4.2761,-55.9836,47,TG Hotel
228,mabaruma,GY,8.2000,-59.7833,74,Monty Brooms Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

hover_cols = ["Country", "City", "Hotel Name", "Lat", "Lng"]

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    alpha = .5,
    scale = 2,
    frame_width = 770,
    frame_height = 550,
    color = "City",
    hover_cols = hover_cols    
)

# Display the map plot_2
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)